In [4]:
import pandas as pd
import json
import re
import os
from datetime import datetime
from process_helper import * 

In [5]:
leaks = pd.read_csv('processed_leaks_1.csv')

In [6]:
leaks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Text         44 non-null     object
 1   terrorism    44 non-null     bool  
 2   security     44 non-null     bool  
 3   espionage    44 non-null     bool  
 4   communalism  44 non-null     bool  
 5   keyword      44 non-null     bool  
 6   response     43 non-null     object
dtypes: bool(5), object(2)
memory usage: 1.0+ KB


In [7]:
leaks.head(5)

,Text,terrorism,security,espionage,communalism,keyword,response
0,Pristina Airport – Possible administrative irr...,False,False,False,False,False,Here is the metadata of the text in the reques...
1,"""An interoffice memorandum providing an “outst...",False,True,False,False,True,Here is the metadata of the text in the reques...
2,"""Description..This is a Secret US National Sec...",False,True,False,False,True,Here is the metadata of the text in JSON forma...
3,"""Tokyo's Climate Change Officials to Continue ...",False,False,False,False,False,"{\n ""metadata"": [\n {\n ""..."
4,"""Description..This is a secret US National Sec...",False,True,False,False,True,Here is the metadata of the text in the reques...


In [8]:
leaks[leaks['response'].isnull() == True]

,Text,terrorism,security,espionage,communalism,keyword,response
17,Finance Assistant 1 claimed he had not noticed...,False,False,False,False,False,NaN


In [9]:
# Function to remove anything that is not within curly braces {}
def extract_within_curly_braces(text):
    """Extract content within the outermost curly braces only."""
    if not isinstance(text, str):  # Ensure input is a string
        text = str(text)
    
    # Extract the content within the outermost curly braces
    match = re.search(r"\{.*\}", text, re.DOTALL)
    if match:
        return match.group(0)
    return ""

# Apply the function to extract content within curly braces in the 'response' column
leaks['response'] = leaks['response'].apply(extract_within_curly_braces)

In [10]:
print(leaks['response'].iloc[0])

{
    "metadata": {
        "earliest_date": "2003",
        "terrorism": false,
        "cyber_security": false,
        "espionage": true,
        "communalism": false,
        "countries": ["Kosovo", "Slovenia"]
    }
}


In [11]:
def extract_date_components(metadata):
    if not isinstance(metadata, str):
        return None, None, None  # Return None for all components if metadata is not a string

    # Regular expressions to capture different date formats
    date_patterns = [
        r"(\b\d{4}-\d{2}-\d{2}\b)",  # Matches YYYY-MM-DD
        r"(\b\d{2}/\d{2}/\d{4}\b)",  # Matches MM/DD/YYYY
        r"(\b\d{4}\b)"               # Matches YYYY
    ]

    for pattern in date_patterns:
        match = re.search(pattern, metadata)
        if match:
            date_str = match.group(0)
            try:
                # Parse full dates (e.g., YYYY-MM-DD, MM/DD/YYYY)
                if "-" in date_str:
                    dt = datetime.strptime(date_str, "%Y-%m-%d")
                elif "/" in date_str:
                    dt = datetime.strptime(date_str, "%m/%d/%Y")
                else:
                    # If only the year is available
                    return int(date_str), None, None
                return dt.year, dt.month, dt.day
            except ValueError:
                continue
    return None, None, None

def extract_countries(metadata):
    if not isinstance(metadata, str):
        return None
    # Regex to identify capitalized words (potential country names)
    country_pattern = r"\b[A-Z][a-z]+\b"
    countries = re.findall(country_pattern, metadata)
    return ", ".join(countries) if countries else None


In [12]:
leaks['countries'] = leaks['response'].apply(extract_countries)
leaks['year'], leaks['month'], leaks['day'] = zip(
    *leaks['response'].apply(extract_date_components)
)

In [13]:
leaks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Text         44 non-null     object 
 1   terrorism    44 non-null     bool   
 2   security     44 non-null     bool   
 3   espionage    44 non-null     bool   
 4   communalism  44 non-null     bool   
 5   keyword      44 non-null     bool   
 6   response     44 non-null     object 
 7   countries    42 non-null     object 
 8   year         31 non-null     float64
 9   month        25 non-null     float64
 10  day          25 non-null     float64
dtypes: bool(5), float64(3), object(3)
memory usage: 2.4+ KB


In [14]:
leaks

,Text,terrorism,security,espionage,communalism,keyword,response,countries,year,month,day
0,Pristina Airport – Possible administrative irr...,False,False,False,False,False,"{\n ""metadata"": {\n ""earliest_date"":...","Kosovo, Slovenia",2003.0,NaN,NaN
1,"""An interoffice memorandum providing an “outst...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...",Kosovo,2001.0,5.0,18.0
2,"""Description..This is a Secret US National Sec...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...","United, States, Japan",NaN,NaN,NaN
3,"""Tokyo's Climate Change Officials to Continue ...",False,False,False,False,False,"{\n ""metadata"": [\n {\n ""...","Japan, China, India, United, States, Europe, E...",NaN,NaN,NaN
4,"""Description..This is a secret US National Sec...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...","Japan, United, States, Australia, Canada, Grea...",2008.0,NaN,NaN
5,"""Japanese Strive to Avoid Damage to U.S. Relat...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...","Japan, United, States",2009.0,7.0,22.0
6,"Until the end of June 2002, responsibility for...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...","Kosovo, Yugoslavia, United, Nations",2001.0,1.0,1.0
7,The EU and Japan were engaged as of early Dece...,False,False,False,False,False,"{\n ""metadata"": [\n {\n ""...","December, Japan, Brussels, Tokyo, Doha, Round",NaN,NaN,NaN
8,A 22 October meeting attended by German Chance...,False,False,False,False,False,"{\n ""metadata"": [\n {\n ""...","Germany, France, Italy, Spain",2011.0,10.0,22.0
9,"Israel has reached out to Europe, including It...",False,False,False,False,False,"{\n ""metadata"": {\n ""earliest_date"":...","Israel, Italy, United, States",2010.0,3.0,13.0


In [15]:
leaks['countries'] = leaks['countries'].fillna('NA').astype(str)

leaks['year'] = leaks['year'].fillna(0).astype(int)
leaks['month'] = leaks['month'].fillna(0).astype(int)
leaks['day'] = leaks['day'].fillna(0).astype(int)

In [16]:
leaks.rename(columns={'Text': 'text'}, inplace=True)

In [17]:
leaks.head()

,text,terrorism,security,espionage,communalism,keyword,response,countries,year,month,day
0,Pristina Airport – Possible administrative irr...,False,False,False,False,False,"{\n ""metadata"": {\n ""earliest_date"":...","Kosovo, Slovenia",2003,0,0
1,"""An interoffice memorandum providing an “outst...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...",Kosovo,2001,5,18
2,"""Description..This is a Secret US National Sec...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...","United, States, Japan",0,0,0
3,"""Tokyo's Climate Change Officials to Continue ...",False,False,False,False,False,"{\n ""metadata"": [\n {\n ""...","Japan, China, India, United, States, Europe, E...",0,0,0
4,"""Description..This is a secret US National Sec...",False,True,False,False,True,"{\n ""metadata"": [\n {\n ""...","Japan, United, States, Australia, Canada, Grea...",2008,0,0


In [18]:
leaks.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44 entries, 0 to 43
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   text         44 non-null     object
 1   terrorism    44 non-null     bool  
 2   security     44 non-null     bool  
 3   espionage    44 non-null     bool  
 4   communalism  44 non-null     bool  
 5   keyword      44 non-null     bool  
 6   response     44 non-null     object
 7   countries    44 non-null     object
 8   year         44 non-null     int32 
 9   month        44 non-null     int32 
 10  day          44 non-null     int32 
dtypes: bool(5), int32(3), object(3)
memory usage: 1.9+ KB


In [ ]:
upload_files_to_s3('bia-datathon-structured', folder_path)